In [13]:
from tensorflow.keras.layers import Conv2D, Convolution2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.preprocessing.image  import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
import warnings
warnings.filterwarnings("ignore")


In [14]:
model = Sequential()

# Conv 01 Layer
model.add(Convolution2D(96, 11,padding = "valid",strides =(4, 4),input_shape = (224, 224, 3),activation = "relu"))

model.add(MaxPooling2D(pool_size = (2, 2),strides = (2, 2),padding="valid"))
model.add(BatchNormalization())



# Conv 02 Layer
model.add(Convolution2D(256, 5,padding = "valid",strides =(1, 1),activation = "relu"))

model.add(MaxPooling2D(pool_size = (2, 2),strides = (2, 2),padding="valid"))
model.add(BatchNormalization())


# Conv 03 Layer
model.add(Convolution2D(384, 3,padding = "valid",strides =(1, 1),activation = "relu"))
model.add(BatchNormalization())


# Conv 04 Layer
model.add(Convolution2D(384, 3,padding = "valid",strides =(1, 1),activation = "relu"))
model.add(BatchNormalization())



# Conv 05 Layer
model.add(Convolution2D(256, 3,padding = "valid",strides =(1, 1),activation = "relu"))

model.add(MaxPooling2D(pool_size = (3, 3),strides = (2, 2),padding="valid"))
model.add(BatchNormalization())


# Flatten Layer
model.add(Flatten())


# Dense 01
model.add(Dense(4096, activation="relu"))
model.add(Dropout(0.2))
model.add(BatchNormalization())


# Dense 02
model.add(Dense(4096, activation="relu"))
model.add(Dropout(0.2))
model.add(BatchNormalization())


# Dense 03
model.add(Dense(1000, activation="relu"))
model.add(Dropout(0.2))
model.add(BatchNormalization())


# Final Layer
model.add(Dense(3, activation="softmax"))

In [15]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 54, 54, 96)        34944     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 27, 27, 96)        0         
_________________________________________________________________
batch_normalization_8 (Batch (None, 27, 27, 96)        384       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 23, 23, 256)       614656    
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 11, 11, 256)       0         
_________________________________________________________________
batch_normalization_9 (Batch (None, 11, 11, 256)       1024      
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 9, 9, 384)        

In [34]:
model.compile(optimizer=SGD(learning_rate=0.001, momentum=0.9, decay = 0.005), 
             loss = "sparse_categorical_crossentropy",
             metrics = ["accuracy"])

In [35]:
datasets_directory = "../input/dogs-and-cats-image-classification/dataset/"

In [36]:
batch_size = 128

In [37]:
train_dataGenerator = ImageDataGenerator(rescale = 1./255,
                                         
                                         shear_range = 0.2, 
                                         zoom_range=0.2,
                                         width_shift_range=0.2,
                                         height_shift_range=0.2, 
                                         fill_mode="nearest"
                                        )

validation_dataGenerator = ImageDataGenerator(rescale = 1./255)


In [38]:
# Load the datasets from the folder
trainDatasets = train_dataGenerator.flow_from_directory(datasets_directory+"/training_set", 
                                                       target_size=(224, 224),
                                                       batch_size=batch_size, 
                                                       class_mode='binary')

validationDatasets = validation_dataGenerator.flow_from_directory(datasets_directory+"/test_set", 
                                                       target_size=(224, 224),
                                                       batch_size=batch_size, 
                                                       class_mode='binary')

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [39]:
classInformation = trainDatasets.class_indices
print("Class Information: {}".format(classInformation))
classes = list(classInformation)
print("Class Names: {}".format(classes))

Class Information: {'cats': 0, 'dogs': 1}
Class Names: ['cats', 'dogs']


In [40]:
classSamplesT = trainDatasets.samples
print("Training Data Samples: {}".format(classSamplesT))
classSamplesV = validationDatasets.samples
print("Validation Data Samples: {}".format(classSamplesV))

Training Data Samples: 8000
Validation Data Samples: 2000


In [41]:
weghtPath = "best_weights_9.hdf5"
checkPoint = ModelCheckpoint(weghtPath, monitor="val_acc", verbose = 1, save_best_only = True, save_weghts_only = True, mode = "max")
callBacksList = [checkPoint]

In [ ]:
history = model.fit(trainDatasets,
                    steps_per_epoch = classSamplesT//batch_size, 
                    validation_data=validationDatasets,
                    epochs=20, 
                    validation_steps=classSamplesV//batch_size, 
                    callbacks=callBacksList)

Epoch 1/20
62/62 [==============================] - 392s 6s/step - loss: 1.4086 - accuracy: 0.4098 - val_loss: 0.9753 - val_accuracy: 0.5427
Epoch 2/20
62/62 [==============================] - 371s 6s/step - loss: 1.1464 - accuracy: 0.4997 - val_loss: 0.9317 - val_accuracy: 0.5583
Epoch 3/20
62/62 [==============================] - 371s 6s/step - loss: 1.0450 - accuracy: 0.5250 - val_loss: 0.8832 - val_accuracy: 0.5599
Epoch 4/20
62/62 [==============================] - 370s 6s/step - loss: 0.9691 - accuracy: 0.5494 - val_loss: 0.8100 - val_accuracy: 0.5995
Epoch 5/20
62/62 [==============================] - 369s 6s/step - loss: 0.9220 - accuracy: 0.5659 - val_loss: 0.7577 - val_accuracy: 0.6281
Epoch 6/20
62/62 [==============================] - 371s 6s/step - loss: 0.8854 - accuracy: 0.5799 - val_loss: 0.7533 - val_accuracy: 0.6359
Epoch 7/20
62/62 [==============================] - 370s 6s/step - loss: 0.8483 - accuracy: 0.5976 - val_loss: 0.7138 - val_accuracy: 0.6594
Epoch 8/20
62